In [29]:
DATA_DIR = '/home/nsuprotivniy/Documents/Работа/OKru/Antispam/data/'

In [30]:
!pip install pyarrow
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import seaborn as sns
import user_agents
%matplotlib inline
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import export_graphviz

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
ids = pq.read_table(DATA_DIR + 'encUserBrowserTable-20180217.parquet').to_pandas()

In [12]:
ids[ids['browserId'].isin(ids[ids['userId'] == 'null']['browserId'])].head()

,userId,browserId
0,null,-8.671834e+18
1,null,-9.595715e+17
2,null,7.449208e+18
4,FCvVK57gTsk40EwQWZVINg==,6.855459e+18
5,pSB7l39kFPcJWSLq1fFEbw==,6.011033e+18


In [11]:
ids['browserId'].isin(ids[ids['userId'[] == 'null']['browserId'])

KeyError: False

In [11]:
-8.67e+18

-8.67e+18

In [43]:
from scipy.sparse import csr_matrix

indptr = np.array([0,2,3,6])
indices = np.array([0,2,2,0,1,2, 7, 8])
data = np.array([1,2,3,4,5,6, 7, 8])
csr_matrix( (data,indices,indptr), shape=(3,3) )

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [38]:
for i in range(0, 6):
    print (indptr[i], indptr[i+1], indices[indptr[i]:indptr[i+1]], data[indptr[i]:indptr[i+1]])
    
    

0 2 [0 2] [1 2]
2 3 [2] [3]
3 6 [0 1 2] [4 5 6]


IndexError: index 4 is out of bounds for axis 0 with size 4

In [46]:
pq.read_table(DATA_DIR + 'botsHTTPRequests-20180217_1718.parquet').to_pandas().head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour
0,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,0hs35/dB/t+uT7u/c71+Vw==,17
1,8.293748e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17
2,7.895490e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,e9iJCewg5Ww1ijKmYKdAyg==,17
3,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518792629926,None,https://ok.ru/web-api/pts/webpush,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17
4,-8.322615e+18,None,XK5THOHUVJHpqGojQAKrMQ==,GET,userMain,None,NAV,1518795472623,userMain,https://ok.ru/,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,f3Z8Sr501bB3SvEgCo+9Rg==,18


In [47]:
pq.read_table(DATA_DIR + 'bots_logins.parquet').to_pandas().head()

,USER_ID,TIMESTAMP
0,590869371434,1518457125557
1,590869371434,1517498068170
2,574328997586,1518814339284
3,580166263980,1518210632394
4,577044419769,1517689664478


In [48]:
pq.read_table(DATA_DIR + 'botsLoginsTSCount.parquet').to_pandas().head()

,TIMESTAMP,count
0,1518800133326,1
1,1518139568414,1
2,1517691368748,1
3,1518875782108,1
4,1519077402143,2


In [49]:
pq.read_table(DATA_DIR + 'botsLogins.parquet').to_pandas().head()

,APP_ID,BROWSER_COOKIE_ID,BROWSER_SIZE,CONFIRMATION,CORRELATION,DEVICE_ID,FLASH_VERSION,IP,IP_CATEGORY,LOGIN_BLOCK_REASON,...,TIMESTAMP,TIME_ZONE,TYPE,USER_AGENT,USER_AGENT_TYPE,USER_AGENT_VERSION,USER_ID,VERIFICATION_METHOD,VERIFICATION_NOT_SUPPORTED,VERIFICATION_REQUESTED
0,0.0,0.000000e+00,0.0,None,100.0,None,0.0,uXXXriZrpm4PCXkTy9Td8w==,None,None,...,1517914669539,NaN,LOGIN,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,0.0,0.0,Po3A7ty1sqkYSqVJS2z8bw==,LOGIN,False,None
1,NaN,NaN,NaN,None,NaN,None,NaN,wEOtBpHSi+Rv9I7eI5aEQw==,None,None,...,1518844492923,NaN,LOGIN,None,NaN,NaN,eTAG/Ei+lEd8tRqyiGpr+w==,LOGIN,None,None
2,0.0,3.997308e+18,0.0,None,100.0,None,0.0,0WcTvpGqxhLDS/uoNsuX2Q==,None,None,...,1517993233860,NaN,LOGIN,User-agent: Mozilla/5.0 (Windows NT 6.1; Win64...,6.0,630323913.0,XQ0uKkml1kzGvHjt6jajGw==,LOGIN,False,None
3,0.0,0.000000e+00,0.0,None,100.0,None,0.0,qO8M/gCA3FbY+s8AuxAKPg==,None,None,...,1517558599276,NaN,LOGIN,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0.0,0.0,fiW1LhT5MNmLlaJB38hKTA==,LOGIN,False,None
4,0.0,0.000000e+00,0.0,None,100.0,None,0.0,9jFEWkvIT1C6BV1fblZjPg==,None,None,...,1519055977689,NaN,LOGIN,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0.0,0.0,83Npol+CqYTt++9fFZVkMg==,LOGIN,False,None


In [8]:
pq.read_table(DATA_DIR + 'encUserBrowserTable-20180217.parquet').to_pandas().head()

,userId,browserId
0,null,-8.671834e+18
1,null,-9.595715e+17
2,null,7.449208e+18
3,7Sn8Y/ATfLC7s8QwxyCU8Q==,-6.314497e+18
4,FCvVK57gTsk40EwQWZVINg==,6.855459e+18


In [9]:
pq.read_table(DATA_DIR + 'usersHTTPRequests-20180217_1718.csv').to_pandas().head()

ArrowIOError: Invalid parquet file. Corrupt footer.

In [ ]:
pq.read_table(DATA_DIR + 'usersHTTPRequests-20180217_1718.parquet').to_pandas().head()

In [ ]:
pq.read_table(DATA_DIR + 'usersLogins.parquet').to_pandas().head()

In [5]:
uid_bid = pq.read_table(DATA_DIR + 'encUserBrowserTable-20180217.parquet').to_pandas().head(1000)
pq.ParquetWriter.write_table(pa.Table.from_pandas(uid_bid), DATA_DIR + 'encUserBrowserTable-20180217_sample1000.parquet')

In [3]:
http_bots = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180217_1718.parquet').to_pandas()
http_users = pq.read_table(DATA_DIR + 'usersHTTPRequests-20180217_1718.parquet').to_pandas()
http_bots['isBot'] = 1
http_users['isBot'] = 0
http_all = http_bots.append(http_users)
pq.write_table(pa.Table.from_pandas(http_all), DATA_DIR + 'allHTTPRequests-20180217_1718.parquet')

In [4]:

t = pd.read_table(DATA_DIR + 'allHTTPRequests-20180217_1718_repart.csv')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [55]:
cols_to_retain = [ 'a', 'list', 'of', 'categorical', 'column', 'names' ]
cat_dict = df[ cols_to_retain ].to_dict( orient = 'records' )

NameError: name 'df' is not defined

In [54]:
df_with_dummies

,a,categorical,column,list,names,of
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,0,1
3,0,1,0,0,0,0
4,0,0,1,0,0,0
5,0,0,0,0,1,0


In [40]:
p = pd.DataFrame()

In [41]:
p = p.append(pd.DataFrame([[1, 2], [3, 4]], columns=["a", "b"]))

In [42]:
p = p.append(pd.DataFrame([[5, 6], [7, 8]], columns=["a", "b"]))

In [43]:
p

,a,b
0,1,2
1,3,4
0,5,6
1,7,8


In [48]:
p.loc[:,p.columns.isin(["a", "c",])]

,a
0,1
1,3
0,5
1,7


In [49]:
p[p.columns.isin(["a", "c",])]

ValueError: Item wrong length 2 instead of 4.

In [33]:
window = 3
shift = 1
nshift = 3

In [37]:
rr = []
for i in range(0, int(window / shift)):
    r = []
    for x in range (0, 10):
        r.append(int((x + i * shift) / window) * nshift + i)
    rr.append(r)
print(rr)

[[0, 0, 0, 3, 3, 3, 6, 6, 6, 9], [1, 1, 4, 4, 4, 7, 7, 7, 10, 10], [2, 5, 5, 5, 8, 8, 8, 11, 11, 11]]


In [39]:
rr = []
for i in range(0, int(window / shift)):
    r = []
    for x in range (0, 10):
        r.append(int(x / shift + 2 * i))
    rr.append(r)
print(rr)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]


In [4]:
!pip install uasparser2

  Running setup.py bdist_wheel for uasparser2 ... done
  Stored in directory: /home/nsuprotivniy/.cache/pip/wheels/4a/e3/aa/e471dce09c54c2d900b263513836bb748a54efe8c2dbcbe94c
Successfully built uasparser2
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
!pip install JPype1
import jpype as jp

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os

# Use pathsep to join JAR files
jp.startJVM(jp.getDefaultJVMPath(), "-Djava.class.path=/home/nsuprotivniy/Projects/Antispam/requirements/yauaa-4.2.jar")

# Another way could be: pd = jp.JClass("org.apache.pdfbox.pdmodel.PDDocument")


# # Use str() to convert to strings, instead of JString
# jp.java.lang.System.out.println("Hello World "+str(document.getNumberOfPages()))

In [10]:
jp.shutdownJVM()

In [3]:
UserAgentAnalyzer = jp.JPackage("nl").basjes.parse.useragent.UserAgentAnalyzer
#UserAgentAnalyzer
uaa = UserAgentAnalyzer.newBuilder()

TypeError: Package nl.basjes.parse.useragent.UserAgentAnalyzer.newBuilder is not Callable

In [ ]:
    UserAgentAnalyzer uaa = UserAgentAnalyzer
                .newBuilder()
                .hideMatcherLoadStats()
                .withCache(25000)
                .build();

    UserAgent agent = uaa.parse("Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11");

    for (String fieldName: agent.getAvailableFieldNamesSorted()) {
        System.out.println(fieldName + " = " + agent.getValue(fieldName));
    }

In [17]:
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=1, input_type='string')
D = ['dog', 'cat', 'eleph', 'Mozilla', 'Windows']
f = h.transform(D)
f.toarray()

array([[ 1.],
       [-1.],
       [ 1.],
       [ 3.],
       [-3.]])

In [26]:
df = pd.DataFrame({'a': np.array([0, 1, 2, 3])})

In [28]:
pa.Table.from_pandas(df)

pyarrow.Table
a: int64
__index_level_0__: int64
metadata
--------
{b'pandas': b'{"index_columns": ["__index_level_0__"], "column_indexes": [{"na'
            b'me": null, "field_name": null, "pandas_type": "unicode", "numpy_'
            b'type": "object", "metadata": {"encoding": "UTF-8"}}], "columns":'
            b' [{"name": "a", "field_name": "a", "pandas_type": "int64", "nump'
            b'y_type": "int64", "metadata": null}, {"name": null, "field_name"'
            b': "__index_level_0__", "pandas_type": "int64", "numpy_type": "in'
            b't64", "metadata": null}], "pandas_version": "0.22.0"}'}

In [35]:
bots = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180217_1718_parsedUAA.parquet').to_pandas()
users = pq.read_table(DATA_DIR + 'usersHTTPRequests-20180217_1718_parsedUAA.parquet').to_pandas()
bots['isBot'] = 1
users['isBot'] = 0
http = users.append(bots).sample(frac=1, random_state=17)
table = pa.Table.from_pandas(http)
pq.ParquetWriter(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA.parquet', table.schema).write_table(table)

In [36]:
bots = pq.read_table(DATA_DIR + 'encAnonymusBotsHTTPRequests-20180217_parsedUAA.parquet').to_pandas()
users = pq.read_table(DATA_DIR + 'encAnonymusUsersHTTPRequests-20180217_parsedUAA.parquet').to_pandas()
bots['isBot'] = 1
users['isBot'] = 0
http = users.append(bots).sample(frac=1, random_state=17)
table = pa.Table.from_pandas(http)
pq.ParquetWriter(DATA_DIR + 'encAnonymusHTTPRequests-20180217_parsedUAA.parquet', table.schema).write_table(table)